Overall plan:
- Get sampled values from image
- Center image on source peak
- 5 arcsec radius?
- Interpolate vertical and horizontal directions to "smooth" image
- Shove into SQL database

In [ ]:
from astropy.io import fits
from find_source import fits_data_index

In [ ]:
fits_file = '' # change this to actual path for testing


##### BELOW IS COPIED FROM region_stats #####
i = fits_data_index(fits_file)

# open FITS file
try:
    file = fits.open(fits_file)
except:
    print(f'Unable to open {fits_file}')

# extract data array
info = file[i]
data = info.data

In [ ]:
data_array = data[0]

In [ ]:
sampled_values = {} # dictionary of sampled data points

Should test to decide how "finely" I want to interpolate things

In [ ]:
def interpolated_value(x):
    # TODO: assert k between -1 and N + 1
    # TODO: find x_k and x_k+1 s.t. x_k <= x <= x_k+1
    # TODO: grab c values from dictionary
    # TODO: plug everything into interpolation function
    # TODO: return value
    pass